## Spatial Modeling and Analytics
### Segment 5 of 5
# Exploration
># A Simple Site Selection Example

<i>Lesson Developers: </i>
<ul>
    <li>
    <i>Karen Kemp (kakemp@usc.edu)</i>
    </li>
    <li>
    <i>Nafiseh Haghtalab (haghtala@msu.edu)</i>
    </li>
    <li>
    <i>Mohsen Ahmadkhani (ahmad178@umn.edu)</i>
    </li>
    <li>
    <i>Joshua Levitt</i>
    </li>
</ul>

In [ ]:
# This code cell starts the necessary setup for Hour of CI lesson notebooks.
# First, it enables users to hide and unhide code by producing a 'Toggle raw code' button below.
# Second, it imports the hourofci package, which is necessary for lessons and interactive Jupyter Widgets.
# Third, it helps hide/control other aspects of Jupyter Notebooks to improve the user experience
# This is an initialization cell
# It is not displayed because the Slide Type is 'Skip'

from IPython.display import HTML, IFrame, Javascript, display
from ipywidgets import interactive
import ipywidgets as widgets
from ipywidgets import Layout

import getpass # This library allows us to get the username (User agent string)

# import package for hourofci project
import sys
sys.path.append('../../supplementary') # relative path (may change depending on the location of the lesson notebook)
import hourofci

# load javascript to initialize/hide cells, get user agent string, and hide output indicator
# hide code by introducing a toggle button "Toggle raw code"
HTML(''' 
    <script type="text/javascript" src=\"../../supplementary/js/custom.js\"></script>
    
    <style>
        .output_prompt{opacity:0;}
    </style>
    
    <input id="toggle_code" type="button" value="Toggle raw code">
''')

# Let's Create a Basic Site Suitability Model

## Goal: 
Find buildings in a city that are suitable candidates for a new coffee shop business.

## Criteria:
The candidate buildings should be:
1. A building type of commercial, retail, or office building
1. At least 400 meters from other coffee shops
1. Close to a bikepath
1. Close to a cinema

## The process:
1. Determine the criteria (done!)
1. Get data
1. Create buffers
1. Assign weights
1. Intersect and sum values

The result is a map showing the site suitability values. Suitability is indicated by the value - high values are highly suitable. 

# Get the data
First, as usual, we need to import the appropriate python packages, with `osmnx` being the most important one since this is where our data come from.

In [ ]:
from IPython import get_ipython
import osmnx as ox 
import pandas as pd
import geopandas as gpd
import folium
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
get_ipython().run_line_magic('matplotlib', 'inline')
#Tells jupyter to plot matplotlib figures inline


Since we're going to call for Minneapolis repeatedly, let's set a variable to store our location. 

[In OSM, you can use standard place names. If you want to run this notebook later for a different place, you can simply put a new placename in here. Remember that since OSM is crowd-sourced, you might not find all the places you want to use in the dataset. However, all major US and global cities are probably there.] 

In [ ]:
place = 'Minneapolis, MN'
place

Our criteria require that we get data about coffee shops, bikepaths, cinemas and buildings. The OSM data contains all these kinds of data, but we have to extract each one separately for our model.

In this first block, we'll get cafes whose cuisine is coffee shop in Minneapolis, Minnesota. 
We use osmnx to create a gdf (GeoDataFrame) which is stored in the `coffee_shops` variable.

In [ ]:
tags = {'amenity':'cafe', 'cuisine':'coffee-shop'}  
coffee_shops = ox.geometries_from_place(place, tags) 

# Convert to UTM
coffee_shops = coffee_shops.to_crs('epsg:3174')[['name', 'geometry']]

coffee_shops.info()
coffee_shops.head()

We got 140 coffee shops, did you? 

And what does this look like?

In [ ]:
coffee_shops.plot()

Next we'll get the bikepaths. Since OSM is crowd-sourced, the tagging of features is often inconsistent. Sometimes you need to use more than one tag to find all the features you are looking for. Here we're using three. 

[If you want to know more about OSM tags for mapped features, see https://wiki.openstreetmap.org/wiki/Map_features.] 

In [ ]:
warnings.filterwarnings("ignore")
tags = {'highway':'cycleway','route':'bicycle','cycleway':True}
bikepaths = ox.geometries_from_place(place, tags)
bikepaths.reset_index(inplace = True)
bikepaths = bikepaths[['osmid', 'highway', 'geometry']]
bikepaths = bikepaths.to_crs('epsg:3174') 
bikepaths.plot()
bikepaths

Now get the cinema point features.

In [ ]:
warnings.filterwarnings("ignore")
tags = {'amenity':'cinema'} 
cinemas = ox.geometries_from_place(place, tags)
cinemas = cinemas.to_crs('epsg:3174') 
cinemas.reset_index(inplace = True)
# Check and see if all objects are of Point type. If not a Point, then replace it by its centroid.
for i, el in cinemas.iterrows():
    if el.geometry.type != 'Point':
        cinemas.iloc[i, 7] = el.geometry.centroid
        
cinemas.plot()
cinemas

Finally fetch the footprints (outlines, i.e. polygons) for commerical, retail and office buildings in Minneapolis. This may take some time, so be patient while waiting for the asterisk to change to a number. 

In [ ]:
tags = {'building':['commercial','retail','offices']}
warnings.filterwarnings("ignore")
buildings = ox.geometries_from_place(place, tags)
buildings = buildings[buildings.geometry.type=='Polygon']
buildings = buildings.to_crs('epsg:3174')[['name', 'type', 'phone', 'building', 'geometry']]
buildings.plot(figsize = (20,10))
buildings.head()

Now, we have all our data. Let's go back to the criteria so we can see how we need to manipulate these data.

Recall that the candidate buildings should be:

1. A building type of commercial, retail, or office building
1. At least 400 meters from other coffee shops
1. Close to a bikepath
1. Close to a cinema

OK, we've already taken care of criteria #1 by getting data about only buildings of these types. To do criteria #2 we need to create buffers...

## Create buffers

Buffers are used to define the area of influence of features. We'll buffer the coffee shops by 400m as an exclusion zone in which we don't want to select candidate sites. 

In [ ]:
coffee_shops_buffer = gpd.GeoDataFrame(coffee_shops.buffer(400), geometry = coffee_shops.buffer(400))
coffee_shops_buffer.plot(figsize = (5,5))

Good. This plot shows all those areas that are within 400m buffer of existing coffee shops. We do not want to include buildings in these areas in our result. 

Now we need to deal with the final two criteria in which locations close to cinemas and bikepaths are more favorable than those that are farther away. Thus places nearby should have higher value in our site selection than places far away - we do this by assigning weights.

## Assign weights

There are many ways to assign weights in site suitability models. Since this is all vector data, we're going to assign weights by creating concentric buffers with declining value as distance from the feature increases. For example, we prefer places that are close to cinemas, so locations that are less than 500 m get a higher weight than places between 500 and 1000 m, and those get more than places 1000 to 1500m away. Anything futher than 1500 gets no weight at all! 

Let's see how this works with our Cinema data.

In [ ]:
# Cinema weighting
warnings.filterwarnings("ignore")
cinema_df1 = gpd.GeoDataFrame(cinemas.buffer(1500), geometry = cinemas.buffer(1500))
cinema_df2 = gpd.GeoDataFrame(cinemas.buffer(1000), geometry = cinemas.buffer(1000))
cinema_df3 = gpd.GeoDataFrame(cinemas.buffer(500), geometry = cinemas.buffer(500))

cinema3 = cinema_df3
cinema3['weight'] = 3

cinema2 = gpd.overlay(cinema_df2, cinema_df3, how='difference')
cinema2['weight'] = 2

cinema1 = gpd.overlay(cinema_df1, cinema_df2, how='difference')
cinema1['weight'] = 1

ax = cinema3.plot(figsize = (10,10), color = 'red')
ax2 = cinemas.plot(ax = ax , color = 'blue')
ax3 = cinema2.plot(ax = ax2, color = '#FF7779') 
cinema1.plot(ax = ax3, color = '#f4c2c2') 


Note how the buffers nest inside of each other. Weights are 3 for the smallest, 2 for the middle one and 1 for the largest/furthest away. 


In [ ]:

warnings.filterwarnings("ignore")
cinema_w = gpd.overlay(cinema1, cinema2, how='union')

cinema_w = gpd.overlay(cinema_w, cinema3, how='union')
# cinema_w.plot(figsize = (10,10))

cinema_w['weights'] = pd.concat([cinema_w['weight_1'].fillna(0).astype('int'), 
                                 cinema_w['weight_2'].fillna(0).astype('int'), 
                                 cinema_w['weight'].fillna(0).astype('int')], axis = 1).max(axis=1)

cinema_w

Now we assign weights to the bikepaths. We'll set only 2 weights - 2 for locations less than 150 m away and 1 for locations between 150 to 300m. 

In [ ]:
warnings.filterwarnings("ignore")
bikepaths_df2 = gpd.GeoDataFrame(bikepaths.buffer(150), geometry = bikepaths.buffer(150))
bikepaths_df1 = gpd.GeoDataFrame(bikepaths.buffer(300), geometry = bikepaths.buffer(300))

bikepaths2 = bikepaths_df2
bikepaths2['weight'] = 2

bikepaths1 = gpd.overlay(bikepaths_df1, bikepaths_df2, how='difference')
bikepaths1['weight'] = 1

bikepaths_w = gpd.overlay(bikepaths1, bikepaths2, how='union')

fig, (ax1,ax2 , ax3) = plt.subplots(ncols=3, sharex=True, sharey=True, figsize=(20,20))
bikepaths1.plot(ax = ax1)
bikepaths2.plot(ax = ax2)
bikepaths_w.plot(ax = ax3)

bikepaths_w['weights'] = pd.concat([bikepaths_w['weight_1'].fillna(0).astype('int'), 
                                 bikepaths_w['weight_2'].fillna(0).astype('int')], axis = 1).max(axis=1)

bikepaths_w



## Intersect and sum values to find the highest value locations

In [ ]:
warnings.filterwarnings("ignore")
res_union1 = gpd.overlay(bikepaths_w, coffee_shops_buffer, how='difference')
res_union = gpd.overlay(res_union1[res_union1.geometry.type=='Polygon'], cinema_w, how='intersection')
# sum up the weights
res_union['final_weights'] = res_union['weights_1'].fillna(0).astype('int') + res_union['weights_2'].fillna(0).astype('int')
res_union = res_union[['final_weights', 'geometry']] # keep only the final_weights and geometry columns
res_union.columns = ['weight', 'geometry'] # rename the columns 
res_union


In [ ]:
res_union.plot(figsize = (10,10), column = 'weight', cmap = 'Reds')


Visualize the final result on a folium interactive map. 

When you hover over each point, you’ll see the area of the footprint of the available building in square meters. 

By clicking on each building you can see its properties like "Type," "Name," and "Phone."

In [ ]:
warnings.filterwarnings("ignore")
sites = gpd.overlay(res_union, buildings, how='intersection')
sites['geo_area'] = sites.area

sites = sites.to_crs(epsg='4326')
sites['centroid'] = sites.centroid
m = folium.Map(location = [44.96, -93.2650], tiles='OpenStreetMap' , zoom_start = 12)


for _, r in sites.iterrows():

    sim_geo = gpd.GeoSeries(r['geometry'])
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j, 
                           style_function = lambda x: {'color': 'red', 'weight': 1,  'fillColor': 'YlGnBu'})
    folium.Popup(f"<i>Type: {r['building']}, Area: {r['geo_area']}</i>").add_to(geo_j)

    
    geo_j.add_to(m)
    
    folium.Marker([r['centroid'].y, r['centroid'].x], popup=f"<i>Type: {r['building']}, Name: {r['name']}, Phone: {r['phone']}</i>", tooltip=f"<i>Area: {r['geo_area']}</i>").add_to(m)


m

# Congratulations!


**You have finished an Hour of CI!**


But, before you go ... 

1. Please fill out a very brief questionnaire to provide feedback and help us improve the Hour of CI lessons. It is fast and your feedback is very important to let us know what you learned and how we can improve the lessons in the future.
2. If you would like a certificate, then please type your name below and click "Create Certificate" and you will be presented with a PDF certificate.

<font size="+1"><a style="background-color:blue;color:white;padding:12px;margin:10px;font-weight:bold;" href="https://forms.gle/JUUBm76rLB8iYppN7">Take the questionnaire and provide feedback</a></font>

In [ ]:

# This code cell loads the Interact Textbox that will ask users for their name
# Once they click "Create Certificate" then it will add their name to the certificate template
# And present them a PDF certificate
from PIL import Image
from PIL import ImageFont
from PIL import ImageDraw

from ipywidgets import interact

def make_cert(learner_name, lesson_name):
    cert_filename = 'hourofci_certificate.pdf'

    img = Image.open("../../supplementary/hci-certificate-template.jpg")
    draw = ImageDraw.Draw(img)

    cert_font   = ImageFont.truetype('../../supplementary/cruft.ttf', 150)
    cert_fontsm = ImageFont.truetype('../../supplementary/cruft.ttf', 80)
    
    _,_,w,h = cert_font.getbbox(learner_name)  
    draw.text( xy = (1650-w/2,1100-h/2), text = learner_name, fill=(0,0,0),font=cert_font)
    
    _,_,w,h = cert_fontsm.getbbox(lesson_name)
    draw.text( xy = (1650-w/2,1100-h/2 + 750), text = lesson_name, fill=(0,0,0),font=cert_fontsm)
    
    img.save(cert_filename, "PDF", resolution=100.0)   
    return cert_filename


interact_cert=interact.options(manual=True, manual_name="Create Certificate")

@interact_cert(name="Your Name")
def f(name):
    print("Congratulations",name)
    filename = make_cert(name, 'Beginner Spatial Modeling and Analytics')
    print("Download your certificate by clicking the link below.")
    
    
    

<font size="+1"><a style="background-color:blue;color:white;padding:12px;margin:10px;font-weight:bold;" href="hourofci_certificate.pdf?download=1" download="hourofci_certificate.pdf">Download your certificate</a></font>
